# Initialize LLMs

In [ ]:
from langchain_ollama import ChatOllama

deepseekcloud_llm = ChatOllama(
    model="deepseek-v3.1:671b-cloud",        
    base_url="http://localhost:11434",
    temperature=0.5
)

In [ ]:
# ✅ Imports (new modular LangChain architecture)
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


# ✅ Define the prompt template
pom_prompt = PromptTemplate(
    input_variables=["page_description"],
    template="""
You are an expert in creating modern, production-grade Playwright Page Object Models in TypeScript.

Generate a TypeScript POM class using Playwright for the page described below.
The structure should follow this pattern, but adapt class name, locators, and methods to fit the content.

Use this structure as a base reference (for example purposes only):

import {{ Page, Locator, expect }} from '@playwright/test'; 

export class PageLogin {{
    readonly page: Page;
    readonly usernameInput: Locator;
    readonly passwordInput: Locator;
    readonly loginButton: Locator;

    constructor(page: Page) {{
        this.page = page;
        this.usernameInput = page.locator('input[name="username"]');
        this.passwordInput = page.locator('input[name="password"]');
        this.loginButton = page.locator('button[type="submit"]:has-text("Login")');
    }}

    async goto(url: string): Promise<void> {{
        await this.page.goto(url);
    }}

    async login(username: string, password: string): Promise<void> {{
        await this.usernameInput.fill(username);
        await this.passwordInput.fill(password);
        await this.loginButton.click();
    }}
}}

---
Now generate a POM class for this page description:

{page_description}

Output only valid TypeScript code (no explanations).
"""
)

# ✅ Load text file (e.g. Docs/LoginPom.txt)
file_path = "./Docs/LoginPom.txt"

with open(file_path, "r", encoding="utf-8") as file:
    page_description = file.read()

# ✅ Combine the pipeline using new Runnable syntax
# (Prompt → LLM → Output Parser)
chain = pom_prompt | deepseekcloud_llm | StrOutputParser()

# ✅ Generate the POM dynamically
generated_pom = chain.invoke({"page_description": page_description})

# ✅ Print result
print("✅ Generated Playwright POM:\n")
print(generated_pom)


### 📝 Generate POM Pattern from text file 